#### Step 1: Create MySQL connection

In [14]:
import toml
import mysql.connector
import pandas as pd
import streamlit as st

# ---- Load secrets ----
secrets_path = r"C:/Users/ayesh/Downloads/mini_project2/.streamlit/secrets.toml"
secrets = toml.load(secrets_path)
db_config = secrets["database"]

# ---- Connect to MySQL ----
connection = mysql.connector.connect(
    host=db_config["db_host"],
    user=db_config["db_user"],
    password=db_config["db_pass"],
    database=db_config["db_name"]
)
cursor = connection.cursor()
print("✅ MySQL connection established!")

✅ MySQL connection established!


In [4]:
cursor.execute('CREATE DATABASE IF NOT EXISTS choco_crunch;')
cursor.execute("USE choco_crunch;")
print("Database ready: choco_crunch")

# function used to write mysql queries in python

Database ready: choco_crunch


In [5]:
df = pd.read_csv("C:/Users/ayesh/Downloads/mini_project2/choco_engineered.csv")

In [6]:
print(df.columns)

Index(['product_code', 'product_name', 'brand', 'energy_kcal', 'energy_kj',
       'carbohydrates', 'sugars', 'fat', 'saturated_fat', 'proteins', 'fiber',
       'salt', 'sodium', 'nova_group', 'nutrition_score_fr',
       'fruits_veg_nuts_pct', 'sugar_to_carb_ratio', 'calorie_category',
       'sugar_category', 'is_ultra_processed'],
      dtype='object')


In [7]:
df.head()

,product_code,product_name,brand,energy_kcal,energy_kj,carbohydrates,sugars,fat,saturated_fat,proteins,fiber,salt,sodium,nova_group,nutrition_score_fr,fruits_veg_nuts_pct,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed
0,6111035000430,Sidi Ali,Sidi Ali,0.0,2.0,4.2,1.4,0.0,0.00000,0.0,0.0,0.000000,0.000000,4.0,1.0,0.0,0.333333,Low Calorie,Low Sugar,Yes
1,6111242100992,perly,perly,97.0,406.0,9.4,4.2,3.0,1.53665,8.0,2.8,0.400000,0.160000,4.0,7.0,0.0,0.446809,Low Calorie,Low Sugar,Yes
2,6111035002175,Sidi Ali,sidi ali,281.0,1243.0,18.0,4.2,7.2,1.53665,6.2,2.8,0.065000,0.026000,1.0,0.0,0.0,0.233333,Moderate Calorie,Low Sugar,No
3,6111035000058,Eau minérale naturelle,"Les Eaux Minérales d'oulmès,Sidi Ali",281.0,1243.0,18.0,4.2,7.2,1.53665,6.2,2.8,0.065000,0.026000,1.0,0.0,0.0,0.233333,Moderate Calorie,Low Sugar,No
4,6111252421568,اكوافينا,AQUAFINA,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000508,0.000203,4.0,0.0,0.0,0.000000,Low Calorie,Low Sugar,Yes


#### Step 2: Create Tables


In [8]:
# Table 1: product_info
cursor.execute("""
CREATE TABLE IF NOT EXISTS product_info (
    product_code VARCHAR(50) PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
""")

# Table 2: nutrient_info
cursor.execute("""
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code VARCHAR(50),
    energy_kcal_value FLOAT,
    energy_kj_value FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    saturated_fat_value FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    fruits_veg_nuts_pct FLOAT,
    nutrition_score_fr INT,
    nova_group INT,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
""")

# Table 3: derived_metrics
cursor.execute("""
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code VARCHAR(50),
    sugar_to_carb_ratio FLOAT,
    calorie_category VARCHAR(20),
    sugar_category VARCHAR(20),
    is_ultra_processed VARCHAR(5),
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
""")

connection.commit()
print("✅ Tables created successfully!")

✅ Tables created successfully!


### Deleting old data for fresh insert

In [9]:
# Delete dependent tables first
cursor.execute("DELETE FROM derived_metrics;")   # child table 2
cursor.execute("DELETE FROM nutrient_info;")     # child table 1
cursor.execute("DELETE FROM product_info;")      # parent table last
connection.commit()
print("✅ Old data deleted for fresh insert")

✅ Old data deleted for fresh insert


### Step 3: Insert Data

In [10]:
insert_product_info = """
INSERT INTO product_info (product_code, product_name, brand)
VALUES (%s, %s, %s)
"""

cursor.executemany(insert_product_info, [
    (r['product_code'], r['product_name'], r['brand']) for _, r in df.iterrows()
])

connection.commit()
print("✅ product_info inserted!")

✅ product_info inserted!


In [11]:
insert_nutrient_info = """
INSERT INTO nutrient_info (
    product_code, energy_kcal_value, energy_kj_value, carbohydrates_value, sugars_value,
    fat_value, saturated_fat_value, proteins_value, fiber_value, salt_value, sodium_value,
    fruits_veg_nuts_pct, nutrition_score_fr, nova_group
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_nutrient_info, [
    (
        r['product_code'], r['energy_kcal'], r['energy_kj'], r['carbohydrates'], r['sugars'],
        r['fat'], r['saturated_fat'], r['proteins'], r['fiber'], r['salt'], r['sodium'],
        r['fruits_veg_nuts_pct'], r['nutrition_score_fr'], r['nova_group']
    ) for _, r in df.iterrows()
])

connection.commit()
print("✅ nutrient_info inserted!")


✅ nutrient_info inserted!


In [12]:
insert_derived_metrics = """
INSERT INTO derived_metrics (
    product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
)
VALUES (%s, %s, %s, %s, %s)
"""

cursor.executemany(insert_derived_metrics, [
    (
        r['product_code'], r['sugar_to_carb_ratio'], r['calorie_category'], 
        r['sugar_category'], r['is_ultra_processed']
    ) for _, r in df.iterrows()
])

connection.commit()
print("✅ derived_metrics inserted!")


✅ derived_metrics inserted!


#### Step 4: Quick sanity check


In [13]:
cursor.execute("SELECT COUNT(*) FROM product_info")
print("Total products in product_info:", cursor.fetchone()[0])

connection.close()
print("✅ MySQL connection closed!")

Total products in product_info: 11997
✅ MySQL connection closed!


# SQL QUERIES

## 📑 product_info 


##### 1. Count products per brand


In [68]:
query = """
SELECT brand, COUNT(*) AS product_count
FROM product_info
GROUP BY brand
ORDER BY product_count DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

# Convert to DataFrame for easier analysis
import pandas as pd
df_brand_counts = pd.DataFrame(results, columns=["brand", "product_count"])
df_brand_counts.head(10)  # show top 10


,brand,product_count
0,Unknown,646
1,Hacendado,391
2,tesco,274
3,Carrefour,147
4,Nestlé,105
5,Bjorg,103
6,Heinz,102
7,Lidl,100
8,Sainsbury's,83
9,Danone,81


##### 2. Count unique products per brand

In [63]:
query = """
SELECT brand, COUNT(DISTINCT product_name) AS unique_products
FROM product_info
GROUP BY brand
ORDER BY unique_products DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

# Optional: convert to DataFrame for easier viewing
import pandas as pd
df_unique_products = pd.DataFrame(results, columns=["brand", "unique_products"])
df_unique_products.head()


,brand,unique_products
0,Unknown,373
1,Hacendado,350
2,Tesco,264
3,Carrefour,134
4,Bjorg,100


##### 3️. Top 5 brands by product count

In [64]:
query = """
SELECT brand, COUNT(*) AS product_count
FROM product_info
GROUP BY brand
ORDER BY product_count DESC
LIMIT 5;
"""

cursor.execute(query)
results = cursor.fetchall()

# Convert to DataFrame for easier analysis
import pandas as pd
df_top5_brands = pd.DataFrame(results, columns=["brand", "product_count"])
df_top5_brands


,brand,product_count
0,Unknown,646
1,Hacendado,391
2,tesco,274
3,Carrefour,147
4,Nestlé,105


##### 4. Products with missing product name

In [65]:
query = """
SELECT *
FROM product_info
WHERE product_name IS NULL OR product_name = '';
"""

cursor.execute(query)
results = cursor.fetchall()

# Convert to DataFrame
import pandas as pd
df_missing_names = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_missing_names


,product_code,product_name,brand


##### 5️. Number of unique brands

In [66]:
query = """
SELECT COUNT(DISTINCT brand) AS unique_brands
FROM product_info;
"""

cursor.execute(query)
result = cursor.fetchone()
print("Total unique brands:", result[0])


Total unique brands: 3401


##### 6️. Products with code starting with '3'

In [67]:
query = """
SELECT *
FROM product_info
WHERE product_code LIKE '3%';
"""

cursor.execute(query)
results = cursor.fetchall()

# Convert to DataFrame for easier viewing
import pandas as pd
df_products_start3 = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_products_start3


,product_code,product_name,brand
0,30000010204,OLD FASHIONED,QUAKER OATS
1,30000010402,OLD FASHIONED,QUAKER OATS
2,30000012000,QUICK 1-MINUTE OATS,QUAKER OATS
3,30000117,Le Pâté Hénaff,Hénaff
4,30000169018,LIGHTLY SALTED RICE CAKES,QUAKER
...,...,...,...
3655,392778,Single Estate Venezuelan Dark clChocolate,Sainsbury's taste the difference
3656,392785,Peruvian Dark Milk Chocolate,Sainsbury's
3657,393645,Taste the Difference vanilla ice cream,"Taste the difference,Sainsbury's"
3658,395496,All Butter Pistachio and Almond Cookies,M&S Food


# 📑 nutrient_info


##### 1. Top 10 products with highest energy-kcal_value


In [78]:
query1 = """
SELECT n.product_code, p.product_name, n.energy_kcal_value
FROM nutrient_info n
JOIN product_info p
ON n.product_code = p.product_code
ORDER BY n.energy_kcal_value DESC
LIMIT 10;
"""

cursor.execute(query1)
results = cursor.fetchall()

import pandas as pd
df_top10_calories = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_top10_calories

,product_code,product_name,energy_kcal_value
0,8480000054647,Semillas de chía,45793.5
1,6111259090132,Pizzarella premium,5280.0
2,8480000505453,Havarti,1580.0
3,8480000208767,Margarina ligera,1548.0
4,6111099000599,Unknown,900.0
5,6111024001516,Huile de table équilibrée riche en vitamines A...,900.0
6,6111024002186,Houilor,900.0
7,6111099000582,Lio huile de table,900.0
8,6111024004746,Huile d'olive vierge de Maroc,900.0
9,6111024001530,Huile Lesieur 3 Graines,900.0


##### 2.  Average sugars_value per nova-group


In [79]:
query2 = """
SELECT nova_group, AVG(sugars_value) AS avg_sugars
FROM nutrient_info
GROUP BY nova_group;
"""
cursor.execute(query2)
results = cursor.fetchall()
df_avg_sugars_nova = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_avg_sugars_nova

,nova_group,avg_sugars
0,4,13.465069
1,3,7.308921
2,1,5.633896
3,2,17.632210


##### 3. Count products with fat_value > 20g

In [80]:
query3 = "SELECT COUNT(*) AS high_fat_products FROM nutrient_info WHERE fat_value > 20;"
cursor.execute(query3)
results = cursor.fetchall()
df_high_fat_count = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_high_fat_count


,high_fat_products
0,3167


##### 4. Average carbohydrates_value per product

In [32]:
query4 = """
SELECT 
    p.product_name AS product_name,
    AVG(n.carbohydrates_value) AS average_carbs_value
FROM product_info p
JOIN nutrient_info n ON n.product_code = p.product_code
GROUP BY p.product_name
ORDER BY average_carbs_value DESC;
"""

cursor.execute(query4)
results = cursor.fetchall()

# Create DataFrame with column names matching aliases
df_avg_carbs = pd.DataFrame(results, columns=['product_name', 'average_carbs_value'])
df_avg_carbs

,product_name,average_carbs_value
0,Semillas de chía,3260.00000
1,Galette maïs chocolat au lait,126.92308
2,Sirop d’agave,105.00000
3,Sucre Cassonnade,100.00000
4,Sucre de glace,100.00000
...,...,...
9727,Eau de table purifiée,0.00000
9728,Mario thon entier,0.00000
9729,Greenwood verveine,0.00000
9730,AïnIfrane,0.00000


##### 5. Products with sodium_value > 1g

In [82]:
query5 = """
SELECT product_code, sodium_value
FROM nutrient_info
WHERE sodium_value > 1;
"""
cursor.execute(query5)
results = cursor.fetchall()
df_high_sodium = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_high_sodium

,product_code,sodium_value
0,50184453,4.320
1,6111184001562,39.600
2,5000157024671,2.360
3,6111184001197,2.000
4,8715035110106,6.760
...,...,...
523,5056053303631,1.300
524,5000328149035,1.144
525,180999001001,1.250
526,5060366570086,4.600


##### 6. Count products with non-zero fruits_veg_nuts_pct

In [83]:
query6 = "SELECT COUNT(*) AS nonzero_fruits_veg_nuts FROM nutrient_info WHERE fruits_veg_nuts_pct > 0;"
cursor.execute(query6)
results = cursor.fetchall()
df_nonzero_fvn = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_nonzero_fvn


,nonzero_fruits_veg_nuts
0,6500


##### 7. Products with energy_kcal_value > 500


In [ ]:
query7 = """
SELECT product_code, energy_kcal_value
FROM nutrient_info
WHERE energy_kcal_value > 500
ORDER BY energy_kcal_value DESC;
"""
cursor.execute(query7)
results = cursor.fetchall()
df_high_energy = pd.DataFrame(results, columns=[col[0] for col in cursor.description])
df_high_energy

,product_code,energy_kcal_value
0,6111099003897,675.0
1,3046920029759,592.0
2,3046920028004,566.0
3,3017620425035,539.0
4,6111031005064,504.0
...,...,...
1870,4066447423761,824.0
1871,767707001678,714.0
1872,7622201768836,558.0
1873,8411414010240,528.0


# 📑 derived_metrics


##### 1. Count products per calorie_category

In [110]:
cursor.execute("""
SELECT calorie_category, COUNT(*) AS count
FROM derived_metrics
GROUP BY calorie_category
""")
df_calorie_count = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_calorie_count

,calorie_category,count
0,Low Calorie,4350
1,Moderate Calorie,4089
2,High Calorie,3558


##### 2. Count of High Sugar products

In [94]:
cursor.execute("""
SELECT COUNT(*) AS high_sugar_count
FROM derived_metrics
WHERE sugar_category = 'High Sugar'
""")
df_high_sugar = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_high_sugar

,high_sugar_count
0,984


##### 3. Average sugar_to_carb_ratio for High Calorie products

In [95]:
cursor.execute("""
SELECT AVG(sugar_to_carb_ratio) AS avg_sugar_to_carb
FROM derived_metrics
WHERE calorie_category = 'High Calorie'
""")
df_avg_sugar_high_cal = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_avg_sugar_high_cal


,avg_sugar_to_carb
0,0.456793


##### 4. Products that are both High Calorie and High Sugar


In [31]:
query = """
SELECT p.product_name, d.product_code
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
WHERE d.calorie_category = 'High Calorie'
AND d.sugar_category = 'High Sugar'
ORDER BY p.product_name ASC;
"""

cursor.execute(query)
results = cursor.fetchall()

# Define column names manually to be safe
columns = ['product_name', 'product_code']

df_high_cal_high_sugar = pd.DataFrame(results, columns=columns)
df_high_cal_high_sugar


,product_name,product_code
0,100G COTE D'OR NOIR ORIGINAL,5410081201039
1,150G LINDOR LAIT LINDT,3046920042963
2,180G AMAN CARA COTE D OR,7622210995025
3,180G C OR LT RAISIN NOISET,7622210995049
4,180G LT NOISETTE COTE D OR,7622210995063
...,...,...
484,Бял шоколад,4025700001962
485,Вафла МОРЕНИ maxx,7622300331283
486,Лешников крем с натрошени лешници и какао,20871765
487,Нутела,80895237


##### 5. Number of products marked as ultra-processed

In [97]:
cursor.execute("""
SELECT COUNT(*) AS ultra_processed_count
FROM derived_metrics
WHERE is_ultra_processed = 'Yes'
""")
df_ultra_processed = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_ultra_processed

,ultra_processed_count
0,7555


##### 6. Products with sugar_to_carb_ratio > 0.7


In [100]:
query = """
SELECT product_code, sugar_to_carb_ratio
FROM derived_metrics
WHERE sugar_to_carb_ratio > 0.7
ORDER BY sugar_to_carb_ratio DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

# Convert to a proper DataFrame
df_sugar_ratio_gt_07 = pd.DataFrame(
    results,
    columns=[col[0] for col in cursor.description]
)

# Optional: reset index for nicer display
df_sugar_ratio_gt_07.reset_index(drop=True, inplace=True)

df_sugar_ratio_gt_07


,product_code,sugar_to_carb_ratio
0,41500007229,8936.170000
1,20002329,42.000000
2,6111184003283,30.000000
3,5060337501316,23.333300
4,8402001021428,21.000000
...,...,...
4136,8423352106947,0.701493
4137,5000225016980,0.701493
4138,4337256381185,0.701299
4139,6111250472500,0.701149


##### 7. Average sugar_to_carb_ratio per calorie_category

In [102]:
cursor.execute("""
SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_sugar_to_carb
FROM derived_metrics
GROUP BY calorie_category
""")
df_avg_sugar_per_calorie = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_avg_sugar_per_calorie

,calorie_category,avg_sugar_to_carb
0,Low Calorie,2.672055
1,Moderate Calorie,0.365609
2,High Calorie,0.456793


# 📑 Join Queries

##### 1. Top 5 brands with most High Calorie products


In [112]:
cursor.execute("""
SELECT p.brand, COUNT(*) AS high_calorie_count
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
WHERE d.calorie_category = 'High Calorie'
GROUP BY p.brand
ORDER BY high_calorie_count DESC
LIMIT 5
""")
df_top_brands_high_cal = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_top_brands_high_cal


,brand,high_calorie_count
0,Hacendado,112
1,Unknown,106
2,Tesco,63
3,Gullón,58
4,Gerblé,51


##### 2. Average energy-kcal_value for each calorie_category


In [113]:
cursor.execute("""
SELECT d.calorie_category, AVG(n.energy_kcal_value) AS avg_energy_kcal
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
GROUP BY d.calorie_category
""")
df_avg_energy_per_category = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_avg_energy_per_category


,calorie_category,avg_energy_kcal
0,Low Calorie,81.206334
1,Moderate Calorie,305.447283
2,High Calorie,549.569822


##### 3. Count of ultra-processed products per brand


In [114]:
cursor.execute("""
SELECT p.brand, COUNT(*) AS ultra_processed_count
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand
""")
df_ultra_per_brand = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_ultra_per_brand

,brand,ultra_processed_count
0,Sidi Ali,2
1,AQUAFINA,2
2,Jaouda,57
3,Ain atlas,1
4,"Marmite,Unilever",1
...,...,...
2322,jus,1
2323,panda,1
2324,"Youngs, Young's",1
2325,Swizzels,1


##### 4. Products with High Sugar and High Calorie along with brand


In [115]:
cursor.execute("""
SELECT p.product_name, p.brand, d.calorie_category, d.sugar_category
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
WHERE d.calorie_category = 'High Calorie' AND d.sugar_category = 'High Sugar'
""")
df_high_cal_high_sugar = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_high_cal_high_sugar


,product_name,brand,calorie_category,sugar_category
0,Nutella,Ferrero,High Calorie,High Sugar
1,Nutella,Ferrero,High Calorie,High Sugar
2,Unknown,Maruja,High Calorie,High Sugar
3,Nocciolata,Rigoni Di Asiago,High Calorie,High Sugar
4,Pâte à tartiner noisettes et cacao,Bonne Maman,High Calorie,High Sugar
...,...,...,...,...
484,Pate à tartiner aux noisettes,LES P'TITES TARTINES,High Calorie,High Sugar
485,KitKat Chunky,Nestlé,High Calorie,High Sugar
486,Dairy Milk Little Bar,Cadbury,High Calorie,High Sugar
487,Cigares fourrés Cacao,Unknown,High Calorie,High Sugar


##### 5. Average sugar content per brand for ultra-processed products


In [116]:
cursor.execute("""
SELECT p.brand, AVG(n.sugars_value) AS avg_sugars
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
JOIN nutrient_info n ON d.product_code = n.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand
""")
df_avg_sugars_ultra = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_avg_sugars_ultra

,brand,avg_sugars
0,Sidi Ali,0.700000
1,AQUAFINA,2.100000
2,Jaouda,7.063158
3,Ain atlas,0.000000
4,"Marmite,Unilever",1.200000
...,...,...
2322,jus,8.500000
2323,panda,49.000000
2324,"Youngs, Young's",0.500000
2325,Swizzels,63.000000


##### 6. Number of products with fruits/vegetables/nuts content in each calorie_category

In [117]:
cursor.execute("""
SELECT d.calorie_category, COUNT(*) AS products_with_fv_nuts
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
WHERE n.fruits_veg_nuts_pct > 0
GROUP BY d.calorie_category
""")
df_fv_nuts_per_category = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_fv_nuts_per_category


,calorie_category,products_with_fv_nuts
0,Moderate Calorie,1901
1,High Calorie,2051
2,Low Calorie,2548


##### 7. Top 5 products by sugar_to_carb_ratio with their calorie and sugar category


In [118]:
cursor.execute("""
SELECT p.product_name, d.sugar_to_carb_ratio, d.calorie_category, d.sugar_category
FROM derived_metrics d
JOIN product_info p ON d.product_code = p.product_code
ORDER BY d.sugar_to_carb_ratio DESC
LIMIT 5
""")
df_top5_sugar_to_carb = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
df_top5_sugar_to_carb

,product_name,sugar_to_carb_ratio,calorie_category,sugar_category
0,Mostassa groga clàssica,8936.1700,Low Calorie,Low Sugar
1,Emmental Milbona,42.0000,Moderate Calorie,Low Sugar
2,Assaisonnement au citron,30.0000,Low Calorie,Low Sugar
3,Bebida energética Ultra Zero,23.3333,Low Calorie,Low Sugar
4,"Gelatina 0% sabor cereza, cola y sandía",21.0000,Low Calorie,Low Sugar


columns in nutreint_info [('product_code', 'varchar(50)', 'YES', 'MUL', None, ''), ('energy_kcal_value', 'float', 'YES', '', None, ''), ('energy_kj_value', 'float', 'YES', '', None, ''), ('carbohydrates_value', 'float', 'YES', '', None, ''), ('sugars_value', 'float', 'YES', '', None, ''), ('fat_value', 'float', 'YES', '', None, ''), ('saturated_fat_value', 'float', 'YES', '', None, ''), ('proteins_value', 'float', 'YES', '', None, ''), ('fiber_value', 'float', 'YES', '', None, ''), ('salt_value', 'float', 'YES', '', None, ''), ('sodium_value', 'float', 'YES', '', None, ''), ('fruits_veg_nuts_pct', 'float', 'YES', '', None, ''), ('nutrition_score_fr', 'int', 'YES', '', None, ''), ('nova_group', 'int', 'YES', '', None, '')] 

columns in derived_metrics [('product_code', 'varchar(50)', 'YES', 'MUL', None, ''), ('sugar_to_carb_ratio', 'float', 'YES', '', None, ''), ('calorie_category', 'varchar(20)', 'YES', '', None, ''), ('sugar_category', 'varchar(20)', 'YES', '', None, ''), ('is_ultra_processed', 'varchar(5)', 'YES', '', None, '')] 


columns in product_info [('product_code', 'varchar(50)', 'NO', 'PRI', None, ''), ('product_name', 'text', 'YES', '', None, ''), ('brand', 'text', 'YES', '', None, '')]